In [50]:
from collections import deque, namedtuple


# we'll use infinity as a default distance to nodes.
inf = float('inf')
Edge = namedtuple('Edge', 'start, end, cost')


def make_edge(start, end, cost=1):
      return Edge(start, end, cost)


class Graph:
    def __init__(self, edges):
        # let's check that the data is right
        wrong_edges = [i for i in edges if len(i) not in [2, 3]]
        if wrong_edges:
            raise ValueError('Wrong edges data: {}'.format(wrong_edges))

        self.edges = [make_edge(*edge) for edge in edges]

    @property
    def vertices(self):
        return set(
            sum(
                ([edge.start, edge.end] for edge in self.edges), []
            )
        )

    def get_node_pairs(self, n1, n2, both_ends=True):
        if both_ends:
            node_pairs = [[n1, n2], [n2, n1]]
        else:
            node_pairs = [[n1, n2]]
        return node_pairs

    def remove_edge(self, n1, n2, both_ends=True):
        node_pairs = self.get_node_pairs(n1, n2, both_ends)
        edges = self.edges[:]
        for edge in edges:
            if [edge.start, edge.end] in node_pairs:
                self.edges.remove(edge)

    def add_edge(self, n1, n2, cost=1, both_ends=True):
        node_pairs = self.get_node_pairs(n1, n2, both_ends)
        for edge in self.edges:
            if [edge.start, edge.end] in node_pairs:
                return ValueError('Edge {} {} already exists'.format(n1, n2))

        self.edges.append(Edge(start=n1, end=n2, cost=cost))
        if both_ends:
            self.edges.append(Edge(start=n2, end=n1, cost=cost))

    @property
    def neighbours(self):
        neighbours = {vertex: set() for vertex in self.vertices}
        for edge in self.edges:
            neighbours[edge.start].add((edge.end, edge.cost))

        return neighbours

    def dijkstra(self, source, dest):
        assert source in self.vertices, 'Such source node doesn\'t exist'
        distances = {vertex: inf for vertex in self.vertices}
        previous_vertices = {
            vertex: None for vertex in self.vertices
        }
        distances[source] = 0
        vertices = self.vertices.copy()

        while vertices:
            current_vertex = min(
                vertices, key=lambda vertex: distances[vertex])
            vertices.remove(current_vertex)
            if distances[current_vertex] == inf:
                break
            for neighbour, cost in self.neighbours[current_vertex]:
                alternative_route = distances[current_vertex] + cost
                if alternative_route < distances[neighbour]:
                    distances[neighbour] = alternative_route
                    previous_vertices[neighbour] = current_vertex

        path, current_vertex = deque(), dest
        while previous_vertices[current_vertex] is not None:
            path.appendleft(current_vertex)
            current_vertex = previous_vertices[current_vertex]
        if path:
            path.appendleft(current_vertex)
        return path


adjacency = []

    
graph = Graph([
    
    ("Painted World", "Anor Londo", 7),  ("Anor Londo", "Duke's Archives", 9),  
    ("Duke's Archives", "Crystal Caves", 14), ("Anor Londo", "Sen's Fortress", 10),
    
    ("Sen's Fortress", "Undead Parish", 15), ("Undead Parish", "Darkroot Garden", 11), 
     ("Undead Parish", "Firelink Shrine", 7),  ("Darkroot Garden", "Darkroot Basin", 9),  
    
    ("Undead Parish", "Upper Burg", 14), ("Tomb of the Giants", "Catacombs", 10),
    
    ("Catacombs", "Firelink Shrine", 15), ("Firelink Shrine", "Undead Asylum", 11), 
    
    ("Firelink Shrine", "Upper Burg", 7),  ("Firelink Shrine", "Upper Londo", 9),  
    
    ("Firelink Shrine", "Firelink Altar", 7),  ("Firelink Altar", "The Kiln", 9),  

    ("Firelink Altar", "The Abyss", 14), ("The Abyss", "Lower Londo", 10),
    
    ("Lower Londo", "Quelaag's Domain", 15), ("Quelaag's Domain", "Upper Demon Ruins", 11), 
    
    ("Quelaag's Domain", "Mid Demon Ruins", 11), ("Quelaag's Domain", "Lower Demon Ruins", 11),
    
    ("Upper Demon Ruins", "Mid Demon Ruins", 11), ("Mid Demon Ruins", "Lower Demon Ruins", 11),
    
    ("Mid Demon Ruins", "Lost Izalith", 11), ("Lower Demon Ruins", "Lost Izalith", 11),
    
    ("Upper Burg", "Lower Burg", 11), ("Upper Burg", "Darkroot Basin", 11),
    
    ("Darkroot Basin", "Oolacile Sanctuary Garden", 11), ("Oolacile Sanctuary Garden", "Oolacile Sanctuary", 11),

    ("Oolacile Sanctuary", "Royal Wood", 11), ("Royal Wood", "Oolacile Township", 11),
    ("Oolacile Township", "Chasm of the Abyss", 11), ("Oolacile Township", "Battle of Stoicism", 11),
    
    ("Firelink Altar", "Lower Londo", 14), ("Lower Londo", "The Depths", 10),
    ("Darkroot Basin", "Valley of Drakes", 14), ("Upper Londo", "Valley of Drakes", 10),

    
    ("Valley of Drakes", "Lower Blighttown", 14), ("Lower Blighttown", "Upper Blighttown", 10),
    
    
    ("The Depths",  "Upper Blighttown", 14), ("Upper Londo", "The Abyss", 10),


    ("Upper Londo",  "Lower Londo", 14), ("Valley of Drakes", "Lower Londo", 10),

    ("Lower Blighttown",  "Quelaag's Domain", 14), ("Lower Blighttown", "Great Hollow", 10),
    
    ("Great Hollow", "Ash Lake", 10)


])

print(graph.dijkstra("Upper Burg", "Ash Lake"))

deque(['Upper Burg', 'Darkroot Basin', 'Valley of Drakes', 'Lower Blighttown', 'Great Hollow', 'Ash Lake'])
